In [6]:
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re

# from Bio import SeqIO

from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio.Blast import NCBIWWW
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIXML
from Bio import SearchIO
import requests,sys,json
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy

## Searching for literature for GLI2 gene

In [7]:
database = "PubMed"
word = 'GLI2'
res= int('30')
email= "rodrigoce9@gmail.com"

Entrez.email= email
handle=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle)
handle.close()
idlist= record['IdList']

handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text")
records = Medline.parse(handle)


for record in records:
    #print(record) 
    #print("PMID:",record.get("PMID","-"))
    print("title:", record.get("TI", "-"))
    #print('abstract:', record.get('AB', '-'))
    print("authors:", record.get("AU", "-"))
    print("source:", record.get("SO", "-"))
    print("")

title: c-Jun phosphorylated by JNK is required for protecting Gli2 from proteasomal-ubiquitin degradation by PGE2-JNK signaling axis.
authors: ['Yang J', 'Wang J', 'Zhang Y', 'Huang W', 'Zhang S', 'Yin P', 'Tan W']
source: Biochim Biophys Acta Mol Cell Res. 2022 Dec 26:119418. doi: 10.1016/j.bbamcr.2022.119418.

title: Expression of Indian hedgehog signaling in murine oviductal infundibulum and its relationship with epithelial homeostasis.
authors: ['Hosotani M', 'Ichii O', 'Namba T', 'Masum MA', 'Nakamura T', 'Hasegawa Y', 'Watanabe T', 'Kon Y']
source: Cell Tissue Res. 2022 Dec 29. doi: 10.1007/s00441-022-03722-w.

title: LOXL2 reduces 5-FU sensitivity through the Hedgehog/BCL2 signaling pathway in colorectal cancer.
authors: ['Qiu Z', 'Qiu S', 'Mao W', 'Lin W', 'Peng Q', 'Chang H']
source: Exp Biol Med (Maywood). 2022 Dec 27:15353702221139203. doi: 10.1177/15353702221139203.

title: Icariin Treatment Rescues Diabetes Induced Bone Loss via Scavenging ROS and Activating Primary Cilia/

## Searching for GLI2 sequence

In [3]:
database = 'nucleotide'
word = 'GLI2 and homo sapiens and Chromosome 2 and not predicted and not unverified '
res= '15'
email= 'rodrigoce9@gmail.com'
Entrez.email= email
handle_search=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle_search)
handle_search.close()
idlist= record['IdList']

In [4]:
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    print(info.id, '-', info.description)
    #print('length of seq:', len(info.seq)) #to check the length of the sequences

NM_001374354.1 - Homo sapiens GLI family zinc finger 2 (GLI2), transcript variant 4, mRNA
NM_001374353.1 - Homo sapiens GLI family zinc finger 2 (GLI2), transcript variant 3, mRNA
NM_001371271.1 - Homo sapiens GLI family zinc finger 2 (GLI2), transcript variant 1, mRNA
NM_005270.5 - Homo sapiens GLI family zinc finger 2 (GLI2), transcript variant 2, mRNA
NM_003743.5 - Homo sapiens nuclear receptor coactivator 1 (NCOA1), transcript variant 1, mRNA
NM_001362950.1 - Homo sapiens nuclear receptor coactivator 1 (NCOA1), transcript variant 4, mRNA
NM_001362952.1 - Homo sapiens nuclear receptor coactivator 1 (NCOA1), transcript variant 5, mRNA
NM_147223.3 - Homo sapiens nuclear receptor coactivator 1 (NCOA1), transcript variant 2, mRNA
NM_001362954.1 - Homo sapiens nuclear receptor coactivator 1 (NCOA1), transcript variant 6, mRNA
NM_001362955.1 - Homo sapiens nuclear receptor coactivator 1 (NCOA1), transcript variant 7, mRNA
NM_147233.2 - Homo sapiens nuclear receptor coactivator 1 (NCOA1), 

The selection of the id has to be done manually as there is no pattern in writing the titles of the queries.\
The id __NG_009030.2__ is the only one where the annotated sequence is in chromosome 2, is not a mRNA and actually is RefSeq. This means that the sequence is being used as a standard for well-characterized genes. So id __NG_009030.2__ will be used from now on.

In [8]:
#The correspondent information of NG_015830.1 was downloaded to a file
import os
Entrez.email = "rodrigoce9@gmail.com"
filename = "NG_009030.2.gb"
if not os.path.isfile(filename):
    net_handle = Entrez.efetch(db="nucleotide", id="NG_009030.2", rettype="gb", retmode="text")
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()

In [9]:
record = SeqIO.read(open("NG_009030.2.gb"), format="genbank")
position=0
record_types={}
for x in record.features:
    record_types[x.type]=record_types.get(x.type,0)+1
for i in range(len(record.features)):
    if record.features[i].type == "CDS":
        position=i
print("The length of the sequence: {}\n".format(len(record.seq)))
print("Type of features: {}\n".format(record_types))
print("Comment from NCBI: {}\n".format(record.annotations["comment"]))
#checking the location of the CDS on the original sequence
print("Location of the CDS on the original sequence: {}".format(record.features[position].location))

The length of the sequence: 263786

Type of features: {'source': 1, 'gene': 1, 'mRNA': 1, 'exon': 14, 'CDS': 1, 'misc_feature': 17}

Comment from NCBI: REVIEWED REFSEQ: This record has been curated by NCBI staff. The
reference sequence was derived from AC018866.9, AC017033.5,
KF510752.1, KF510212.1 and AC016764.8.
This sequence is a reference standard in the RefSeqGene project.
On Sep 14, 2022 this sequence version replaced NG_009030.1.
Summary: This gene encodes a protein which belongs to the C2H2-type
zinc finger protein subclass of the Gli family. Members of this
subclass are characterized as transcription factors which bind DNA
through zinc finger motifs. These motifs contain conserved H-C
links. Gli family zinc finger proteins are mediators of Sonic
hedgehog (Shh) signaling and they are implicated as potent
oncogenes in the embryonal carcinoma cell. The protein encoded by
this gene localizes to the cytoplasm and activates patched
Drosophila homolog (PTCH) gene expression. It is al

In [11]:
CDS_nuc_seq=0
CDS_nuc_seq=range(int(record.features[position].location.start), int(record.features[position].location.end))

In [12]:
#saving the CDS_nucleotides and CD_aminoacid seqs in files
filename_CDS_nucl = "CDS_nucleot_GLI2_seq.fasta"
filename_CDS_prot = "CDS_prot_GLI2_seq.fasta"
input_handle  = open(filename, "r")
output_handle_nucl = open(filename_CDS_nucl, "w")
output_handle_prot = open(filename_CDS_prot, "w")
for seq_record in SeqIO.parse(input_handle, "genbank") :
    output_handle_nucl.write(">\n%s" % (CDS_nuc_seq))
    output_handle_prot.write(">\n%s" % ("".join(record.features[position].qualifiers['translation']))) #without join, output is a list
    
output_handle_nucl.close()
output_handle_prot.close()
input_handle.close()

## BLASTN - for all organisms

In [ ]:
record = SeqIO.read(open("10001bp_sequence_GLI2.gb"), format="gb") 
print (len(record.seq))-

In [ ]:
Blast = NCBIWWW.qblast("blastn", "nt", record.seq, entrez_query = "Homo Sapiens[organism]")

In [ ]:
with open('blast_DNA_GLI2_file.xml', "w") as out_handle:
        out_handle.write(Blast.read())
Blast.close()

In [13]:
results_Blastn = open("blast_DNA_GLI2_file.xml")
blastn_records = NCBIXML.read(results_Blastn)
for parameter in blastn_records.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)


Accession:  NG_009030
Definition:  Homo sapiens GLI family zinc finger 2 (GLI2), RefSeqGene on chromosome 2
E-value:  0.0
E-value:  1.23303e-94
E-value:  2.71593e-90
E-value:  4.03079e-88
E-value:  1.71394e-86
E-value:  5.98222e-86
E-value:  2.54371e-84
E-value:  8.87841e-84
E-value:  8.87841e-84
E-value:  8.87841e-84
E-value:  3.09887e-83
E-value:  3.77519e-82
E-value:  4.59913e-81
E-value:  4.59913e-81
E-value:  1.60525e-80
E-value:  5.60289e-80
E-value:  2.38241e-78
E-value:  1.23412e-75
E-value:  4.30749e-75
E-value:  4.30749e-75
E-value:  1.50346e-74
E-value:  1.50346e-74
E-value:  1.50346e-74
E-value:  5.2476e-74
E-value:  6.39289e-73
E-value:  6.39289e-73
E-value:  2.23134e-72
E-value:  2.23134e-72
E-value:  9.48789e-71
E-value:  1.40813e-68
E-value:  3.78187e-44
E-value:  3.78187e-44
E-value:  5.61279e-42
E-value:  5.61279e-42
E-value:  3.54206e-38
E-value:  5.9981e-29
E-value:  3.54519e-19
E-value:  4.31892e-18
E-value:  4.04506e-12
E-value:  4.92789e-11
E-value:  6.0034e-10


In [14]:
E_VALUE_THRESH = 0.001
results_Blast= open('blast_DNA_GLI2_file.xml')
blast_records = NCBIXML.read(results_Blast)
for alignment in  blast_records.alignments:
    print (alignment)
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence: ', alignment.title)
            print('lenght:', alignment.length)
            print(hsp.query[0:75] + '...')
            print(hsp.match[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')
            print()
            

gi|2301838502|ref|NG_009030.2| Homo sapiens GLI family zinc finger 2 (GLI2), RefSeqGene on chromosome 2
           Length = 263786

****Alignment****
sequence:  gi|2301838502|ref|NG_009030.2| Homo sapiens GLI family zinc finger 2 (GLI2), RefSeqGene on chromosome 2
lenght: 263786
TTGATCCATTTGGAGTTAATTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAG...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
TTGATCCATTTGGAGTTAATTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAG...

****Alignment****
sequence:  gi|2301838502|ref|NG_009030.2| Homo sapiens GLI family zinc finger 2 (GLI2), RefSeqGene on chromosome 2
lenght: 263786
TTAATTTTTTT-TTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTC...
||||||  ||| |||||||||||||||||||| |||  ||||||||||| ||||||| | | |||||| ||||||...
TTAATTAATTTATTTTTTTGAGACAGAGTCTCGCTCCATCACCCAGGCTGGAGTGCAGTGGCGCCATCATGGCTC...

****Alignment****
sequence:  gi|2301838502|ref|NG_009030.2| Homo sapiens GLI family zinc finger 

sequence:  gi|1909942459|dbj|AP023478.1| Homo sapiens DNA, chromosome 18, nearly complete genome
lenght: 77846715
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCT-GTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTG...
||||||||||||||||||  |||| || |||| ||| |||||||| ||||||||| |||| ||||||||||||||...
TTTTTTTTTTTTTTGAGATGGAGTTTCACTCTTGTCGCCCAGGCTGGAGTGCAATGGTGCAATCTTGGCTCACTG...

****Alignment****
sequence:  gi|1909942459|dbj|AP023478.1| Homo sapiens DNA, chromosome 18, nearly complete genome
lenght: 77846715
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
||||||||||||||||||| ||||||| ||||||| |||||||| ||||||| | | || |||| ||||||||||...
TTTTTTTTTTTTTTGAGACGGAGTCTCGCTCTGTCGCCCAGGCTGGAGTGCAGTGGCGCGATCTCGGCTCACTGC...

****Alignment****
sequence:  gi|1909942459|dbj|AP023478.1| Homo sapiens DNA, chromosome 18, nearly complete genome
lenght: 77846715
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCA-TCTTGGCTCACTG...
|| |||||||||||||||||||||||| ||||| |||||||||| ||||||| | || ||| |||||

lenght: 77846715
GTTAATTTTTTTTTTTTTTGAGACAGAGTCTCCCTCT-GTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCT...
||||| ||| ||||||||||||||||||| ||  ||| ||  |||||||| || |||||| | |  ||||  |||...
GTTAACTTTCTTTTTTTTTGAGACAGAGTTTCGTTCTTGTTGCCCAGGCTGGACTGCAATGGCGTGATCTCAGCT...

****Alignment****
sequence:  gi|1909942459|dbj|AP023478.1| Homo sapiens DNA, chromosome 18, nearly complete genome
lenght: 77846715
TTAATTTTTTTTTTTTTT-GAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTC...
|||||||||||||||||| ||||| ||||||| |||| |  | |||||  ||||||| | | |  ||||||||||...
TTAATTTTTTTTTTTTTTTGAGACGGAGTCTCGCTCTTTGGCTCAGGCCGGAGTGCAGTGGCGTGATCTTGGCTC...

****Alignment****
sequence:  gi|1909942459|dbj|AP023478.1| Homo sapiens DNA, chromosome 18, nearly complete genome
lenght: 77846715
TTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGCA...
|||||||||  ||||| ||||||||| | ||||| |||||||| ||||||| | |||| ||||||||||||  ||...
TTTTTTTTTAATTGAGGCAGAGTCTCACCCTGTCTCCCAGGCTGGAGTGCAGTGGTGCTATCTTGGCTCACCACA...

****A

****Alignment****
sequence:  gi|2033714914|gb|CP068260.2| Homo sapiens isolate CHM13 chromosome 18
lenght: 80542538
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
||||||||||||||||||||||||||| |||||||||||||||| ||||||| | |||| | || ||||||||||...
TTTTTTTTTTTTTTGAGACAGAGTCTCACTCTGTCACCCAGGCTGGAGTGCAGTGGTGCAACCTCGGCTCACTGC...

****Alignment****
sequence:  gi|2033714914|gb|CP068260.2| Homo sapiens isolate CHM13 chromosome 18
lenght: 80542538
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
||||||||||||||||||| ||||||| |||||| | ||||||| ||||||||| |||| |||||||||||||||...
TTTTTTTTTTTTTTGAGACGGAGTCTCGCTCTGTTATCCAGGCTGGAGTGCAAT-GTGCAATCTTGGCTCACTGC...

****Alignment****
sequence:  gi|2033714914|gb|CP068260.2| Homo sapiens isolate CHM13 chromosome 18
lenght: 80542538
ATTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTG...
|||||||||||||||||||   ||||| |  |||||||||||||| ||||||||| |  | |||| |||||||||...
ATTTTTTTTTTTTTTGAG


****Alignment****
sequence:  gi|2033714914|gb|CP068260.2| Homo sapiens isolate CHM13 chromosome 18
lenght: 80542538
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
||||||||| ||||||||  ||||||| ||||||||||||| || ||||||||| |||  |||| ||||||||||...
TTTTTTTTTCTTTTGAGATGGAGTCTCACTCTGTCACCCAGCCTGGAGTGCAATGGTGTGATCTCGGCTCACTGC...

****Alignment****
sequence:  gi|2033714914|gb|CP068260.2| Homo sapiens isolate CHM13 chromosome 18
lenght: 80542538
TTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGCAA...
||||||||||||||||| || | || | ||||| |||||||| ||||||||| |    ||||  |||||||||||...
TTTTTTTTTTTTGAGACGGAATGTCACCCTGTCGCCCAGGCTGGAGTGCAATGGCATGATCTCAGCTCACTGCAA...

****Alignment****
sequence:  gi|2033714914|gb|CP068260.2| Homo sapiens isolate CHM13 chromosome 18
lenght: 80542538
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCT-GTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTG...
||||||||||||||||||| |||| || |||| |||||||||||| ||||||||| | || ||||  ||||||||...
TTTTTTTTTTTTTTGAG

...

lenght: 94690957
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCT-GTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTG...
|||||||||||||||||||||||| || |||| ||  |||||||| ||||||||| |||| |||||||||||| |...
TTTTTTTTTTTTTTGAGACAGAGTTTCGCTCTTGTTGCCCAGGCTGGAGTGCAATGGTGCAATCTTGGCTCACCG...

****Alignment****
sequence:  gi|1909942457|dbj|AP023476.1| Homo sapiens DNA, chromosome 16, nearly complete genome
lenght: 94690957
TAATTTTTTTTTTTTTTGAGACAGAGTCTCCCTCT-GTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCA...
||||||||||||||||||||||||||| |  |||| ||  |||||||| |||| |||| |||| |||||||||||...
TAATTTTTTTTTTTTTTGAGACAGAGTTTTGCTCTTGTTGCCCAGGCTGGAGTACAATGGTGCGATCTTGGCTCA...

****Alignment****
sequence:  gi|1909942457|dbj|AP023476.1| Homo sapiens DNA, chromosome 16, nearly complete genome
lenght: 94690957
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
||||||||||||||||||| ||||||  | ||||  |||||||| ||||||| | |||| |||||||||||||||...
TTTTTTTTTTTTTTGAGACGGAGTCTTGCCCTGTTGCCCAGGCTGGAGTGCAGTGGTGCGATCTTGGCTCACTGC...

****A

TTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGCAACA...
|||| |||||||||||||||||| |||| || |||||||| ||||||| | |  | |||| ||||||||||||| ...
TTTTGTTTTTGAGACAGAGTCTCGCTCTATCGCCCAGGCTGGAGTGCATTGGCACGATCTCGGCTCACTGCAACC...

****Alignment****
sequence:  gi|1909942457|dbj|AP023476.1| Homo sapiens DNA, chromosome 16, nearly complete genome
lenght: 94690957
TTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGCA...
|||||||||||||||||   |||||   ||||||||  ||||| ||||||||  |  | ||||||||||||||||...
TTTTTTTTTTTTTGAGATGAAGTCTTGTTCTGTCACT-AGGCTGGAGTGCAACGGCACGATCTTGGCTCACTGCA...

****Alignment****
sequence:  gi|1909942457|dbj|AP023476.1| Homo sapiens DNA, chromosome 16, nearly complete genome
lenght: 94690957
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
||||||||||||||||| | ||||||| |||||||||||||||| ||||| | | | ||||||| ||||||||||...
TTTTTTTTTTTTTTGAGGCGGAGTCTCGCTCTGTCACCCAGGCTGGAGTGTAGTGGCGCCATCTCGGCTCACTGC...

****Alignment****
sequ

TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCT-GTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTG...
|||| ||||| |||  ||| ||||||| |||| ||  |||||||| ||||||||| | || | ||  ||||||||...
TTTTGTTTTTGTTTTTGACGGAGTCTCGCTCTTGTTCCCCAGGCTGGAGTGCAATGGCGCAACCTCAGCTCACTG...

****Alignment****
sequence:  gi|1909942457|dbj|AP023476.1| Homo sapiens DNA, chromosome 16, nearly complete genome
lenght: 94690957
TTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGCA...
||| | |||||||||||||||||||  | |  |  ||||||||  |||||||| |||  ||||||||||||||||...
TTTGTATTTTTTTGAGACAGAGTCTTGCGCCATTGCCCAGGCTGCAGTGCAATGGTGTAATCTTGGCTCACTGCA...

****Alignment****
sequence:  gi|1909942457|dbj|AP023476.1| Homo sapiens DNA, chromosome 16, nearly complete genome
lenght: 94690957
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
|||||||||||||||||||| |||||| ||||||  |||||||| |||| |||| |||  |||| ||||||||||...
TTTTTTTTTTTTTTGAGACAAAGTCTCACTCTGTTGCCCAGGCTGGAGTACAATGGTGTGATCTGGGCTCACTGC...

****Alignment****
sequ


****Alignment****
sequence:  gi|16596520|gb|AC008870.8| Homo sapiens chromosome 16 clone CTD-2196E14, complete sequence
lenght: 150522
TTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCA-----AT-----AGTGCCATCTTG----...
||| ||| ||| |||||||||| ||| || ||||||||| |||||||     ||     | ||| || |||    ...
TTTCTTTCTGAAACAGAGTCTCACTCAGT-ACCCAGGCTGGAGTGCATGGGCATGACTCACTGC-ATTTTGAGGT...

****Alignment****
sequence:  gi|16596520|gb|AC008870.8| Homo sapiens chromosome 16 clone CTD-2196E14, complete sequence
lenght: 150522
TTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGC...
||||||||||||||||||||| ||||| || ||| |       || |||||| | |||  | | | |||||||||...
TTTTTTTTTTTTTTGAGACAGGGTCTCACTGTGTTA-------TACAGTGCAGTGGTGTGACCATAGCTCACTGC...

****Alignment****
sequence:  gi|16596520|gb|AC008870.8| Homo sapiens chromosome 16 clone CTD-2196E14, complete sequence
lenght: 150522
TTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCAT--CT-TGGCTCACT...
|||||  |||||||||| || |||||  |||||  ||||

TAATTTTTTTTTTTTTTGAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCAC...
||||||||||||||||| |||||   |||  ||||||| || ||||| ||||||| | |  | ||||||||||||...
TAATTTTTTTTTTTTTT-AGACAAGATCTTGCTCTGTCGCC-AGGCTGGAGTGCAGTGGCACGATCTTGGCTCAC...

****Alignment****
sequence:  gi|1024846813|ref|NG_046987.1| Homo sapiens DEAD-box helicase 10 (DDX10), RefSeqGene on chromosome 11
lenght: 282906
TTTTTTTTTGAGACAGAGTCTCCCTCT-GTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACTGCAACA...
|||||||||||||  |||| |  |||| ||  |||||||| ||||||||| |  | ||||  |||||| ||| | ...
TTTTTTTTTGAGATGGAGTTTTGCTCTTGTTGCCCAGGCTGGAGTGCAATGGCACGATCTCCGCTCACCGCAGCC...

****Alignment****
sequence:  gi|1024846813|ref|NG_046987.1| Homo sapiens DEAD-box helicase 10 (DDX10), RefSeqGene on chromosome 11
lenght: 282906
TTTTTTTTTTTTTT--GAGACAGAGTCTCCCTCTGTCACCCAGGCTAGAGTGCAATAGTGCCATCTTGGCTCACT...
||||||||||||||  ||||  ||||||| || |||| |||||||| ||||||| | | || | |||||||||||...
TTTTTTTTTTTTTTCCGAGATGGAGTCTCGCTGTGTCGCCCAGGCTGGAGTGCAGTGGCGCGAACTTGGCTCAC

In [17]:
#filtering the "predicted" alignments
results_Blastn= open('blast_DNA_GLI2_file.xml')
blastn_records = NCBIXML.read(results_Blastn)
E_VALUE_THRESH = 0.001
count_preditc, count_homo=0,0
list_filtered_alignments=[]
for alignment in  blastn_records.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            exist = re.search(r'PREDICTED:\s', alignment.title)
            if exist:
                # print( existe[0] )
                pre = re.match(r'PREDICTED:\s', exist[0] )
                if pre:
                    count_preditc+=1
                    #print(id)
            else:
                list_filtered_alignments.append(alignment.accession)
                homo=re.search(r'Homo\ssapiens',alignment.title)
                if homo:
                    count_homo+=1
print(list_filtered_alignments)
print('Total {} PREDICTED seqs found and remaining {} ids are from Homo sapiens'.format(count_preditc, count_homo))
#counting of Homo sapiens it's not conclusive, just to have an ideia, because the titles don't follow any pattern

['NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'NG_009030', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'AC016764', 'NM_001374354', 'NM_001374354', 'NM_001374354', 'NM_001374354', 'NM_001374353', 'NM_001374353', 'NM_001374353', 'NM_001374353', 'NM_001371271', 'NM_001371271', 'NM_00

In [18]:
with open('CDS_GLI2_nucleotide_result_blast.txt', 'w') as f:
    for line in list_filtered_alignments:
        f.write(f"{line}\n")

## BLASTP

In [ ]:
record = SeqIO.read(open("10001bp_sequence_GLI2.gb"), format="gb") 
print (len(record.seq))

In [ ]:
Trans=[]
record = SeqIO.read(open("10001bp_sequence_GLI2.gb"), format="gb") 
for feat in record.features:
    if feat.type == "CDS":
        Trans.append(str(feat.qualifiers['translation']))
        print(feat.qualifiers['translation'])

In [ ]:
result_handle = NCBIWWW.qblast("blastp", "swissprot", Trans)

In [ ]:
with open('blast_PROT_GLI2_file.xml', "w") as out_handle:
        out_handle.write(result_handle.read())
result_handle.close()

In [16]:
results_Blastp = open("blast_PROT_GLI2_file.xml")
blastp_records = NCBIXML.read(results_Blastp)
for parameter in blastp_records.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)


Accession:  P10070
Definition:  RecName: Full=Zinc finger protein GLI2; AltName: Full=GLI family zinc finger protein 2; AltName: Full=Tax helper protein [Homo sapiens]
E-value:  0.0

Accession:  Q0VGT2
Definition:  RecName: Full=Zinc finger protein GLI2; AltName: Full=Tax helper protein [Mus musculus]
E-value:  0.0

Accession:  Q91661
Definition:  RecName: Full=Zinc finger protein GLI4; AltName: Full=Neural-specific DNA-binding protein xGLI4; Short=xGLI-4 [Xenopus laevis]
E-value:  0.0

Accession:  Q9IA31
Definition:  RecName: Full=Transcriptional activator GLI3; AltName: Full=GLI3 full-length protein; Short=GLI3FL; Contains: RecName: Full=Transcriptional repressor GLI3R; AltName: Full=GLI3 C-terminally truncated form [Gallus gallus]
E-value:  0.0
E-value:  1.2649e-18

Accession:  P55879
Definition:  RecName: Full=Zinc finger protein GLI2 [Gallus gallus]
E-value:  0.0


In [19]:
results_Blastp= open('blast_PROT_GLI2_file.xml')
blastp_records = NCBIXML.read(results_Blastp)
E_VALUE_THRESH = 0.001
list_filtered_alignments,list_species=[],[]
for alignment in  blastp_records.alignments:
    for hsp in alignment.hsps:
#         print(hsp.identities)    # maybe add more 
        if hsp.expect < E_VALUE_THRESH:
            list_filtered_alignments.append(alignment.accession)
            title_organism=re.search(r'\[.+\s.+\]', alignment.title)
            if title_organism:
                m = re.match(r'\[.+\s.+\]', title_organism[0] )
                specie = m.group(0)
                #print(specie)
                list_species.append(specie)
for x in sorted(set(list_species)):
    print("number of times: {} that appeared specie: {}".format(list_species.count(x),x))

print(list_filtered_alignments)

number of times: 5 that appeared specie: [Bos taurus]
number of times: 1 that appeared specie: [Caenorhabditis briggsae]
number of times: 1 that appeared specie: [Caenorhabditis elegans]
number of times: 4 that appeared specie: [Danio rerio]
number of times: 3 that appeared specie: [Drosophila melanogaster]
number of times: 1 that appeared specie: [Drosophila yakuba]
number of times: 5 that appeared specie: [Gallus gallus]
number of times: 21 that appeared specie: [Homo sapiens]
number of times: 14 that appeared specie: [Mus musculus]
number of times: 2 that appeared specie: [Pan troglodytes]
number of times: 17 that appeared specie: [Xenopus laevis]
number of times: 6 that appeared specie: [Xenopus tropicalis]
['P10070', 'Q0VGT2', 'Q91661', 'Q9IA31', 'Q9IA31', 'P55879', 'Q91660', 'Q91660', 'P10071', 'P10071', 'Q5IS56', 'Q5IS56', 'Q61602', 'Q61602', 'Q91690', 'Q91690', 'P55878', 'P08151', 'P47806', 'P47806', 'P19538', 'Q8NEA6', 'Q8NEA6', 'Q6XP49', 'Q6XP49', 'Q8NBF1', 'Q8K1M4', 'P34708'

In [20]:
with open('CDS_GLI2_protein_result_blastp.txt', 'w') as f:
    for line in list_filtered_alignments:
        f.write(f"{line}\n")

## Uniprot search of Blastp results

In [23]:
#saving sequences
results_Blastp= open('blast_PROT_GLI2_file.xml')
blastp_records = NCBIXML.read(results_Blastp)
titles_list=[]
for alignment in  blastp_records.alignments:   
    titles_list.append(alignment.title)

file= open("CDS_GLI2_protein_result_blastp.txt", "r")
fields="sequence"
WEBSITE_API="https://rest.uniprot.org"
seqs=[]

for i in file:
    r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i, fields))
    seqs.append(str(r.content))

with open('allOrg_CDS_GLI2_prot_new.fasta', 'w') as f:
    for index, seq in enumerate(seqs):
        existe = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL)
        if existe:
            m = re.match( r'b\'Sequence\\n(.+?(?=\\n\'))', seq, re.DOTALL )
            f.write(f">{titles_list[index]}\n{m.group(1)}\n\n")                    
            #print(m.group(1) )       

IndexError: list index out of range

In [22]:
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [ ]:
file= open("CDS_GLI2_protein_result_blastp.txt", "r")
fields="accession,organism_name,protein_name,cc_subcellular_location,cc_function"
WEBSITE_API="https://rest.uniprot.org"
with open('uniprot_result_CDS_filtered.txt', 'w',encoding='utf-8') as f:
    for i in file:
        r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i, fields))
        print(r.text)
        f.write(r.text)
        f.write('\n')